<a href="https://colab.research.google.com/github/al-khabib/sewer-3d-reconstruction/blob/experiment%2FMVS/mvs_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os

# --- Configuration ---
FRAME_SKIP = 15         # Process every Nth frame pair
MIN_MATCH_COUNT = 10    # Minimum good matches needed for pose estimation

# --- Camera Intrinsics (IMPORTANT: Replace with actual values if known!) ---
# Using placeholder values. Accurate intrinsics are crucial.
image_width = 640       # Replace with your video's width if known
image_height = 480      # Replace with your video's height if known
fx = image_width        # Placeholder focal length x
fy = image_width        # Placeholder focal length y
cx = image_width / 2    # Placeholder principal point x
cy = image_height / 2   # Placeholder principal point y

K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0, 0, 1]])

print("--- Configuration ---")
print(f"FRAME_SKIP: {FRAME_SKIP}")
print(f"MIN_MATCH_COUNT: {MIN_MATCH_COUNT}")
print("Camera Matrix (K - Using Placeholders):")
print(K)
print("\nWARNING: Using placeholder camera intrinsics (K). Pose estimation accuracy depends heavily on correct values.")
print("-" * 20)

def detect_and_match_features(img1_gray, img2_gray):
    """Detects ORB features, matches them, and filters using Lowe's ratio test."""
    orb = cv2.ORB_create(nfeatures=1000)
    kp1, des1 = orb.detectAndCompute(img1_gray, None)
    kp2, des2 = orb.detectAndCompute(img2_gray, None)

    if des1 is None or des2 is None or len(des1) < 2 or len(des2) < 2:
        print("  Not enough descriptors found.")
        return None, None, None

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    if matches:
        try:
            for pair in matches:
                 if len(pair) == 2: # Ensure we have k=2 results
                      m, n = pair
                      if m.distance < 0.75 * n.distance: # Lowe's ratio test
                           good_matches.append(m)
        except ValueError:
             print("  Warning: Issue during ratio test, possibly due to insufficient matches.")


    print(f"  Keypoints: Frame 1: {len(kp1)}, Frame 2: {len(kp2)}")
    print(f"  Good matches (Ratio Test): {len(good_matches)}")
    return kp1, kp2, good_matches

def estimate_pose(kp1, kp2, good_matches, K):
    """Estimates relative pose using the Essential Matrix approach."""
    if not good_matches or len(good_matches) < MIN_MATCH_COUNT:
        print(f"  Not enough good matches ({len(good_matches) if good_matches else 0}) to estimate pose. Need {MIN_MATCH_COUNT}.")
        return None, None, None

    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Find Essential Matrix
    E, mask_e = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)

    if E is None:
        print("  Essential Matrix estimation failed.")
        return None, None, None

    num_inliers_e = np.sum(mask_e) if mask_e is not None else 0
    print(f"  Essential Matrix estimated with {num_inliers_e} inliers.")

    if num_inliers_e < MIN_MATCH_COUNT:
         print(f"  Number of inliers ({num_inliers_e}) too low. Pose estimate unreliable.")
         return None, None, mask_e # Return mask even if unreliable

    # Recover Pose
    _, R, t, mask_pose = cv2.recoverPose(E, pts1, pts2, K, mask=mask_e)

    if R is None or t is None:
        print("  Pose recovery (R, t) failed.")
        return None, None, mask_e

    num_inliers_pose = np.sum(mask_pose) if mask_pose is not None else 0
    print(f"  Pose recovered with {num_inliers_pose} final inliers.")
    print(f"  Estimated Translation (t - direction vector):\n{t.flatten()}")

    # Return the mask from findEssentialMat as it corresponds to the input good_matches indices
    return R, t, mask_e

# --- Main Processing ---
if __name__ == "__main__":
    video_path = input("Enter the path to the sewer video file: ")

    if not os.path.exists(video_path):
        print(f"Error: Video file not found at '{video_path}'")
        exit()

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        exit()

    frame_count = 0
    prev_frame_gray = None
    prev_kp = None
    orb = cv2.ORB_create(nfeatures=1000) # Initialize ORB detector once

    while True:
        ret, frame = cap.read()
        if not ret:
            print("\nEnd of video reached or error reading frame.")
            break

        # Optional: Resize frame if needed
        # frame = cv2.resize(frame, (image_width, image_height))
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Process frame pairs based on FRAME_SKIP
        if frame_count > 0 and frame_count % FRAME_SKIP == 0 and prev_frame_gray is not None:
            print(f"\n--- Processing frames {frame_count - FRAME_SKIP} and {frame_count} ---")

            # 1. Detect and Match Features
            kp1, kp2, good_matches = detect_and_match_features(prev_frame_gray, frame_gray)

            if kp1 is None or kp2 is None or good_matches is None:
                 print("  Feature detection/matching failed for this pair.")
                 # Update previous frame and continue
                 prev_frame_gray = frame_gray.copy()
                 prev_kp, _ = orb.detectAndCompute(prev_frame_gray, None)
                 frame_count += 1
                 continue

            # 2. Visualize All Good Matches
            img_matches = cv2.drawMatches(prev_frame_gray, kp1, frame_gray, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            cv2.imshow(f'Good Matches: Frames {frame_count - FRAME_SKIP} & {frame_count}', img_matches)
            print("  Displaying matches (Press any key in the window to continue)...")
            cv2.waitKey(0) # Wait for user input before continuing

            # 3. Estimate Pose
            R, t, pose_mask = estimate_pose(kp1, kp2, good_matches, K)

            # 4. (Optional) Visualize Inlier Matches consistent with Pose
            if pose_mask is not None and good_matches:
                 # Create a mask for drawing inliers based on findEssentialMat mask
                 draw_mask = pose_mask.ravel() == 1
                 # Filter good_matches using the mask
                 # Ensure mask length matches good_matches length
                 if len(draw_mask) == len(good_matches):
                      inlier_matches = [m for i, m in enumerate(good_matches) if draw_mask[i]]
                      print(f"\n  Visualizing {len(inlier_matches)} matches consistent with estimated pose:")
                      img_inlier_matches = cv2.drawMatches(prev_frame_gray, kp1, frame_gray, kp2, inlier_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                      cv2.imshow(f'Inlier Matches (Pose): Frames {frame_count - FRAME_SKIP} & {frame_count}', img_inlier_matches)
                      print("  Displaying inlier matches (Press any key)...")
                      cv2.waitKey(0)
                 else:
                      print(f"  Warning: Mismatch between pose mask length ({len(draw_mask)}) and good matches ({len(good_matches)}). Cannot visualize inliers.")


            print("-" * 10) # Separator for the next pair


        # Update previous frame info for the next suitable iteration
        prev_frame_gray = frame_gray.copy()
        # We could detect prev_kp here, but detect_and_match does it, so maybe not needed

        frame_count += 1

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("\nProcessing finished.")